# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686615


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:  10%|▉         | 3/31 [00:07<01:05,  2.35s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:46,  1.71s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:32,  1.26s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:22,  1.10it/s]

...

Rendering models:  81%|████████  | 25/31 [00:09<00:00,  6.53it/s]

Rendering models:  87%|████████▋ | 27/31 [00:09<00:00,  7.35it/s]

Rendering models:  94%|█████████▎| 29/31 [00:09<00:00,  7.66it/s]

Rendering models:  97%|█████████▋| 30/31 [00:09<00:00,  8.00it/s]

not-logged-in-20a275b53abec8db9f41    5.023909
firefighterwolf13                     0.000426
CThomas                               0.000358
cschwefl                              0.000742
06r2d206                              0.005144
Dynamatt                              0.000212
jmfranci                              0.000655
tosnyder                              0.000840
jdebn537                              0.000837
not-logged-in-9cdd68bb509752635e0b    0.004039
AlexYoung35                           0.000301
TRISTANM18                            0.000384
Lavadude                              0.002611
jordanrushworth                       0.001524
clars915                              0.000283
harmonioushope                        0.000295
not-logged-in-4375b453e37780852499    0.000420
ShirleyL                              0.000292
flowers_in_chains                     0.000276
Gatto_Volante                         0.000525
kgenere                               0.000329
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())